### read in random sample (10%) of full df_kickstarter to work with

In [314]:
import pandas as pd
import os
import random
filename = 'raw_data/df_kickstarter.csv'

p = 1  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(filename, header=0,  skiprows=lambda i: i>0 and random.random() > p)
len(df)

625544

#Categorization

In [315]:
import numpy as np
import json

reserve "Cat_Name"(encoding), "backer_count", "blurb"(for NLP),"converted_pledged_amount","country"(encoding),"launched_at"(convert to time),"currency"(encoding),"deadline"(convert to time),"goal","pledged"(delete the ones pledged>goal?),"staff_pick"(meaning?,encoding), "state"(encoding),"usd_pledged","name"(for NLP)

Don't reserve: "Location" is too noisy, too many 1s, "splotlight" is the same as "state","state_changed_at" almost equals to "deadline" 

In [316]:
#extract category
cat_json = df['category'].apply(json.loads)
cat_name=list(map(lambda d: d['slug'], cat_json))
df['Cat_Name']=cat_name

In [317]:
#only study the success and failure cases
df=df.loc[(df['state'] == 'successful') | (df['state'] =='failed')]

In [318]:
#convert the launched_at column and deadline column to months. Add a column to calculate the period of funding
df['launched_at'] = pd.to_datetime(df['launched_at'],unit='s')
df['deadline'] = pd.to_datetime(df['deadline'],unit='s')

In [319]:
df['Launch_deadline_days']=df['deadline']-df['launched_at']
df['Launch_deadline_days'] = df['Launch_deadline_days'].apply(lambda x: x.days)

In [320]:
df['launched_at'] = df['launched_at'].dt.strftime('%m')
df['deadline'] = df['deadline'].dt.strftime('%m')

In [321]:
#df=df.df.dropna(subset=["location"])
#loc_list=list(map(lambda d:json.loads(d)["name"], df['location']))

In [322]:
#df['Loc_Name']=loc_list

In [323]:
features_dict=dict() #inquiry for the actual value represented by the numerical value

In [324]:
#feature engineering for categorical features
for col in df.columns:
    if col in ['Cat_Name', 'country', 'currency', 'staff_pick','state','launched_at','deadline']:
        df[col], indicators = pd.factorize(df[col])
        features_dict[col]=indicators

In [336]:
reserve=['Launch_deadline_days','Cat_Name', 'country', 'currency', 'staff_pick','state',"backers_count","blurb","goal","launched_at","pledged","deadline","name"]

In [337]:
df=df[reserve]

In [338]:
df

,Launch_deadline_days,Cat_Name,country,currency,staff_pick,state,backers_count,blurb,goal,launched_at,pledged,deadline,name
0,30,0,0,0,0,0,108,"In the year 2025, a young man's superintellige...",4000.0,0,5265.82,0,A Date in 2025 - A sci-fi comedy short film
1,44,1,0,0,1,0,37,Replacement Place needs your support to combat...,5000.0,1,5225.00,1,Replacement Place | New York Live Arts
2,14,2,0,0,0,0,27,Pumpkin Spice Love - A Pumpkin Spice 4 Life 30...,250.0,0,362.00,2,Pumpkin Spice Love!
3,30,3,0,0,0,0,184,Single mama and her 4-year old sail with all-f...,15000.0,2,15678.00,3,Pirate Mama
4,30,4,1,1,0,0,169,SparKit wishes to produce kit-set Wimshurst Ma...,2500.0,3,11412.00,4,SparKit - Miniature Electrostatic Generator
5,18,5,2,2,0,1,1,This funding is esential to completing a gradu...,1000.0,4,1.00,5,Researching potential for protected areas in G...
6,40,6,0,0,0,1,1,Everyday for the next 40 days I will be creati...,5000.0,5,12.00,6,ILS Creative Abstract Digital Portraits!
7,30,5,3,3,0,0,10,Me& Magazine is a quarterly world culture/arts...,500.0,6,601.00,7,Me& Magazine.
8,30,7,0,0,0,0,26,The revolutionary hair scalp stimulator made f...,7500.0,7,8160.00,5,KESHIN
9,17,6,0,0,0,1,2,I am a graphic design major. For my senior the...,600.0,8,100.00,7,The Anglophilia Project


In [339]:
df.shape

(579140, 13)

In [334]:
features_dict["country"][8]

'AU'

In [329]:
df['state'].value_counts() #check the value distribution of a column

0    353150
1    225990
Name: state, dtype: int64

In [330]:
df.dtypes #check the data types of all columns

Launch_deadline_days          int64
Cat_Name                      int64
country                       int64
currency                      int64
staff_pick                    int64
state                         int64
backers_count                 int64
blurb                        object
converted_pledged_amount      int64
goal                        float64
launched_at                   int64
pledged                     float64
usd_pledged                 float64
deadline                      int64
name                         object
dtype: object

In [331]:
#(df['Launch_deadline_days']<0).values.any() #check whether there is funding period < 0

In [332]:
#df.isnull().values.any() #check whether the dataframe has null

In [333]:
#df['current_currency'].unique() #check the the number of different values in a column

Machine Learning Models to Use: Linear Regression, KNN, Decision Tree/Random Forest, PCR/PLS
Measure Metrics: RMSE for regression, Accuracy for classification(pledged > goal, then successful), or more...